In [2]:
# import the necessary packages
from imageSearch.centroidtracker import CentroidTracker
from imageSearch.trackableobject import TrackableObject
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2

 imutils.video  will help us to work with a webcam and to calculate the estimated Frames Per Second (FPS) throughput rate   
 dlib  library will be used for its correlation tracker implementation.